# Data analysis

In this lesson we treat Spatial Data Analysis, often called [Spatial Analysis](https://en.wikipedia.org/wiki/Spatial_analysis)
or even Spatial Statistics. 

> Spatial analysis or spatial statistics includes any of the formal techniques which 
> study entities using their topological, geometric, or geographic properties. From [Wikipedia](https://en.wikipedia.org/wiki/Spatial_analysis). 

Wikipedia actually provides [a nice overview](https://en.wikipedia.org/wiki/Spatial_analysis) to give you a feel what Spatial Analysis is about.


## Analysis with Fiona and Shapely
Let's explore some of Shapely's functions for *(Topological) Spatial Relationships*.
See simplified view below, more [on Wikipedia](https://en.wikipedia.org/wiki/Spatial_relation).

![(Topological) Spatial Relationships](images/spatialrelations.png)
*Examples of topological spatial relations - [By Krauss - Own work, CC BY-SA 3.0](https://commons.wikimedia.org/w/index.php?curid=21299138)*

If you want to dive into
a theoretical background, check out the [Dimensionally Extended nine-Intersection Model (DE-9IM)](https://en.wikipedia.org/wiki/DE-9IM).

We'll start simple, answering the question: **Does the Paraná float through Argentina?**. We'll use two National Earth datasets: Rivers and lakes (LineStrings) and (Admin-0) Countries. In spatial relationship-terms: *Is  the (Multi)LineString representing the Paraná crossing the (Multi)Polygon representing Argentina?"*

We will follow these steps:

* read the Rivers and Lakes dataset
* extract the Paraná feature Geometry  
* read the Countries dataset
* extract the Argentina feature Geometry  
* apply the Shapely `crosses` function
* advanced: list all DE-9IM relations of the two geometries
* advanced: through which countries does the Paraná float?


In [79]:
import fiona
from shapely.geometry import shape

### Fetch the Paraná geometry.


In [80]:
with fiona.open("../data/rivers_lake_centerlines.gpkg") as rivers_lakes:
    for feature in rivers_lakes:
        if feature['properties']['name'] == 'Paraná':
            parana_geom = shape(feature["geometry"])
            print(f'Found Paraná! geom type: {parana_geom.geom_type}')
            break

Found Paraná! geom type: LineString


### Fetch the Argentina geometry

In [81]:
with fiona.open("../data/countries.json") as countries:
    for feature in countries:
        if feature["properties"]["NAME"] == "Argentina":
            argentina_geom = shape(feature["geometry"])
            print(f'Found Argentina! geom type: {argentina_geom.geom_type}')
            break


Found Argentina! geom type: MultiPolygon


### Does the Paraná float through Argentina?


In [82]:
parana_geom.crosses(argentina_geom)


True

### List the (DE-9IM) relationship

In [83]:
parana_geom.relate(argentina_geom)

'101FF0212'

### Through which countries does the Paranà float?


In [84]:
print("The Paranà floats through:")

with fiona.open("../data/countries.gpkg") as countries:
    for feature in countries:
        country_geom = shape(feature["geometry"])
        if parana_geom.crosses(country_geom):
            print(feature["properties"]["NAME"])


The Paranà floats through:
Argentina
Brazil
Paraguay


# Analyzing atmospheric total ozone data

The [World Ozone and Ultraviolet Radiation Data Centre (WOUDC)](https://woudc.org) is one of six World Data Centres which are part of the Global Atmosphere Watch programme of the World Meteorological Organization. WOUDC provides an archive of daily total atmospheric ozone in a given column from the surface to the edge of the atmosphere.  Data is collected from a global network of monitoring stations across the world.

Total ozone is measured in [Dobson units](https://en.wikipedia.org/wiki/Dobson_unit) with an acceptable range of 100 - 700 du.

Let's generate a time series plot based on downloading GeoJSON data (via [WFS](https://geo.woudc.org/ows?service=WFS&version=1.1.0&request=GetFeature&outputformat=GeoJSON&typename=totalozone&filter=%3Cogc:Filter%3E%3Cogc:And%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Eplatform_id%3C/PropertyName%3E%3CLiteral%3E493%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Eplatform_type%3C/PropertyName%3E%3CLiteral%3ESTN%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3Cogc:PropertyIsBetween%3E%3Cogc:PropertyName%3Einstance_datetime%3C/ogc:PropertyName%3E%3Cogc:LowerBoundary%3E1924-01-01%2000:00:00%3C/ogc:LowerBoundary%3E%3Cogc:UpperBoundary%3E2021-12-31%2023:59:59%3C/ogc:UpperBoundary%3E%3C/ogc:PropertyIsBetween%3E%3C/ogc:And%3E%3C/ogc:Filter%3E&sortby=instance_datetime%20DESC&startindex=0&maxfeatures=200000)) of the [CNRS Rio Gallegos station 493](https://woudc.org/data/stations/?id=493&lang=en) record of the data and do some basic range checking.

In [85]:
from datetime import datetime
import json
import gzip

with gzip.open('../data/totalozone-station-493.json.gz') as gzfh:
        data = json.load(gzfh)
 
len(data['features'])

5521

In [86]:
# setup graph axes
x_axis = [datetime.strptime(x['properties']['instance_datetime'], '%Y/%m/%d 00:00:00+00') for x in data['features']]
y_axis = [float(x['properties']['daily_columno3']) for x in data['features']]

# average
total_average = sum(y_axis) / float(len(y_axis))
total_average

304.8217714182213

In [87]:
# render simple plot

from bokeh.plotting import figure, output_notebook, show

# output graph to notebook
output_notebook()

p = figure(title='Bucharest 226 dobson', x_axis_type='datetime', y_axis_label='Dobson units')
p.line(x_axis, y_axis, legend_label="Dobson units", line_width=2)
show(p)

Loading BokehJS ...

Let's test whether any values fall outside the acceptable range:

In [88]:
max(y_axis) > 700 or min(y_axis) < 100

False

Now let's try importing all [WOUDC stations](https://woudc.org/data/stations/) into [GeoPandas](https://geopandas.org) and do some further analysis.  First, display stations as a table grouped by WMO region:

In [89]:
import geopandas

gdf = geopandas.read_file('../data/woudc-stations.geojson')

gdf.groupby('wmo_region').wmo_region.count()

wmo_region
I                 34
II                56
III               37
IV               153
V                 32
VI               191
the Antarctic     34
Name: wmo_region, dtype: int64

Now let's plot them onto a map

In [90]:
p = figure(tools='pan, wheel_zoom', x_axis_type='mercator', y_axis_type='mercator', width=800, height=500)


# We need to make x,y columns for Bokeh to display
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y
        
gdf['x'] = gdf.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
gdf['y'] = gdf.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)
gdf = gdf.drop('geometry', axis=1).copy()

from bokeh.models import ColumnDataSource
psource = ColumnDataSource(gdf)

# add populated places point overlay
p.scatter(x='x', y='y', size=10, alpha=0.7, source=psource, color='blue', legend_label='WMO WOUDC stations')
# add background tiles layer from CARTO
p.add_tile("CartoDB Positron", retina=True)

show(p)

Let's plot only region III stations (i.e. South America)

In [91]:
gdf2 = gdf[gdf.wmo_region.isin(['III']) == True]
psource2 = ColumnDataSource(gdf2)
p.scatter(x='x', y='y', size=10, alpha=0.7, source=psource2, color='red', legend_label='WMO WOUDC stations in WMO Region III')
show(p)

---
[<- Raster Data](05-raster-data.ipynb) | [Visualization ->](07-visualization.ipynb)
